**RL-QVO**

This is the first subgraph matching algorithm using machine learning. (And also my first reading paper about subgraph matching).

A good separation of subgraph matching algorithm is that we could divide this task into three parts. **1. Candidate set generation, 2. Matching order generation, 3. Enumeration procedure.** And as far as I know, most of the subgraph matching paper follows this procedure.

Subgraph matching has been proved to be a **NP-hard** problem, so what we could only do is to **reduce the search space** of the matching procedure.

First, for each nodes in the query graph, we could find the **candidate set** in the graph data, so we don't have to try to match each node in the graph data with each node in the query graph(usually, graph data contains a large number of nodes). 

Then, to make the search space even more less, we should find a **relatively optimal path** of the query graph to search in the graph data.

Finally, we should use some proper algoithm to **search the path before** in the graph data to get the matched subgraph in it.

As we could see from the procedure, the first step and third step need both **accuracy**(we could not miss a possible candidate or a possible subgraph) and **speed**. In this case, at least for now, in my opinion, we could only use machine learning technique in the **second step** like the author did.

I mainly focus on the machine learning part of this paper. The other part is from an algorithm called "Hybrid". Long story short, they use the Candidate set generation from "GraphQL" and use Enumeration procedure from "QuickSI". Besides, they use "RI" as the baseline of the Matching order generation part.

They changed RI part to the machine learning part. First, they use GCN to learn the embedding of the query graph and use softmax to calculate the probability to **select one unselected node as current node in the matching order**. For each step t(generate embedding for nodes and get a unselected node), they calculate **two different rewards**, $r_val$ and $r_h$ to measure this step. After getting a complete matching order(using all the nodes in the query graph to form a path), they use **Enumeration procedure** to get subgraph in the graph data. And they use the number of enumeration of RI as the baseline, so using enumeration of RL-QVO to substract that of RI would be a **new reward** item(here the author use log algorithm to make the total rewards not unbalanced). Since they use **policy gradient** reinforcement learning as the RL part, they calculate loss for the GCN model based on the rewards. 

I've contacted the author, and he said that in this paper, they only consider same number of nodes in the query graph in a train set to make sure the length of the path and the number of the step of RL the same(I reproduce differently by including different length of nodes). And about the performance of the model, the author didn't answer me clearly. He said, and I quoted "I don't know how I make it work, what I only do is adjusting the parameters". Hence, I'm really confused with this paper. Not to mention the log altorithm of the $r_enum$ he gave me was wrong at first.....But still, this is a good idea to explore in the field of subgraph matching

In [ ]:
!git clone https://github.com/RapidsAtHKUST/SubgraphMatching.git

Cloning into 'SubgraphMatching'...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (278/278), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 278 (delta 9), reused 260 (delta 3), pack-reused 0
Receiving objects: 100% (278/278), 2.51 MiB | 14.86 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda

cpu = 1
if cpu:
  CUDA = "cpu"
else:
  CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cpu.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cpu.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cpu.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cpu.html


In [ ]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from torch_geometric.nn import GCNConv
from collections import defaultdict
from copy import deepcopy
import random

seed = 123

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
class graph():
  def __init__(self, graphid, node2label, node2degree, edges, edge_index):
    self.graphid = graphid
    self.node2label = node2label
    self.node2degree = node2degree
    self.edges = edges
    self.edge_index = edge_index
    self.candidateset = defaultdict(set)
    self.label2node = defaultdict(set)
    for node in self.node2label:
      self.label2node[self.node2label[node]].add(node)
    self.phi = []
    self.phiparent = {}
    self.embeddings = {}

  def reset(self):
    self.candidateset = defaultdict(set)
    self.phi = []
    self.phiparent = {}
    self.embeddings = {}
    
  def init_embedding(self, alphadegree, alphad, alphal, g):
    self.embeddings[0] = torch.zeros(len(self.node2label), 7)
    for node in self.node2degree:
      self.embeddings[0][node][0] = self.node2degree[node] / alphadegree
      self.embeddings[0][node][1] = self.node2label[node]
      self.embeddings[0][node][2] = node
      for v in g.node2degree:
        if self.node2degree[node] < g.node2degree[v]:
          self.embeddings[0][node][3] += 1
        if self.node2label[node] == g.node2label[v]:
          self.embeddings[0][node][4] += 1
      self.embeddings[0][node][3] /= len(g.node2degree) * alphad
      self.embeddings[0][node][4] /= len(g.node2degree) * alphal

In [ ]:
def get_graph(filepath, filename):
  node2label = {}
  node2degree = {}
  edges = defaultdict(set)
  edge_index = []
  f = open(filepath, "r", encoding="utf-8")

  _, nodenum, edgenum = f.readline().strip().split()
  for i in range(int(nodenum)):
    _, nodeid, nodelabel, nodedegree = f.readline().strip().split()
    node2label[int(nodeid)] = int(nodelabel)
    node2degree[int(nodeid)] = int(nodedegree)  
  for i in range(int(edgenum)):
    _, node1, node2 = f.readline().strip().split()
    edges[int(node1)].add(int(node2))
    edges[int(node2)].add(int(node1))
    edge_index.append([int(node1), int(node2)])
    edge_index.append([int(node2), int(node1)])

  f.close()
  edge_index = torch.tensor(edge_index).t()
  g = graph(filename, node2label, node2degree, edges, edge_index)

  return g

In [ ]:
def graphQL(q, g, level):
  marks = set()
  for qnode in q.node2label:
    label = q.node2label[qnode]
    q.candidateset[qnode] |= g.label2node[label]
    for gnode in q.candidateset[qnode]:
      marks.add((qnode, gnode))
  
  for i in range(level):
    for qnode in q.node2label:
      for gnode in q.candidateset[qnode].copy():
        mark = (qnode, gnode)
        if mark not in marks:
          continue
        
        b = set()
        qneighbors = q.edges[qnode]
        gneighbors = g.edges[gnode]
        for qneighbor in qneighbors:
          for gneighbor in gneighbors:
            if gneighbor in q.candidateset[qneighbor]:
              b.add(qneighbor)
              break
        
        semiperfect = qneighbors - b
        if len(semiperfect) == 0:
          marks.remove(mark)
        else:
          q.candidateset[qnode].remove(gnode)
          for qneighbor in qneighbors:
            for gneighbor in gneighbors:
              if gneighbor in q.candidateset[qneighbor]:
                mark = (qneighbor, gneighbor)
                marks.add(mark)

    if len(marks) == 0:
      break

In [ ]:
def RI(q):
  maxdegree = 0
  maxnode = 0
  visited = set()
  for node in q.node2degree:
    if maxdegree < q.node2degree[node]:
      maxdegree = q.node2degree[node]
      maxnode = node
  visited.add(maxnode)
  q.phi.append(maxnode)
  q.phiparent[maxnode] = -1
  while len(visited) != len(q.node2degree):
    m = len(q.phi)
    um = -1
    urank = (-1, -1, -1)
    for node in q.node2degree:
      if node in visited:
        continue
      v1 = visited & q.edges[node]
      v2 = set()
      for vis in visited:
        for vode in q.node2degree:
          if node == vode or vode in visited:
            continue
          if vode in q.edges[vis] and vode in q.edges[node]:
            v2.add(vis)
            break
      v3 = set()
      for vode in q.node2degree:
        flag = True
        if node == vode or vode in visited:
          continue            
        for vis in visited:
          if vode in q.edges[vis]:
            flag = False
            break
        if flag:
          v3.add(vode)
      rank = (len(v1), len(v2), len(v3))
      if urank <= rank:
        um = node
        urank = rank
    for parent in q.phi:
      if parent in q.edges[um]:
        q.phiparent[um] = parent
        break
    q.phi.append(um)
    visited.add(um)

In [ ]:
def QuikSI(q, g, m, i, totalresult): # not equal to the original code
  global count
  count += 1
  if i == len(q.phi) + 1:
    totalresult.append(m.copy())
    return 
  result = {}
  u = -1
  for node in q.phi:
    if node not in m:
      u = node
      break

  lc = set()

  if i == 1:
    vs = g.label2node[q.node2label[u]]
    for v in vs:
      if g.node2degree[v] >= q.node2degree[u]:
        lc.add(v)
  else:
    for v in g.edges[m[q.phiparent[u]]]:
      if q.node2label[u] == g.node2label[v] and g.node2degree[v] >= q.node2degree[u]:
        flag = True
        for node in q.phi:
          if node == u:
            break
          if node == q.phiparent[u]:
            continue
          #if m[node] not in g.edges[v]:
          if v == m[node] or (m[node] not in g.edges[v] and (node in q.edges[u] or u in q.edges[node])):
            flag = False
            break
        if flag:
          lc.add(v)

  for node in lc:
    if node not in set(m.values()):
      m[u] = node
      QuikSI(q, g, m, i + 1, totalresult)
      m.pop(u)

In [ ]:
import os
qs = []
qdir = "SubgraphMatching/test/query_graph"
for f in os.listdir(qdir):
  filepath = os.path.join(qdir, f)
  qs.append(get_graph(filepath, f))

gs = []
gdir = "SubgraphMatching/test/data_graph"
for f in os.listdir(gdir):
  filepath = os.path.join(gdir, f)
  gs.append(get_graph(filepath, f))

print(len(qs))
print(len(gs))

f = open("SubgraphMatching/test/expected_output.res", "r", encoding="utf-8")
lines = f.readlines()
f.close()

expects = {}
for line in lines:
  name, times = line.strip().split(":")
  expects[name + ".graph"] = int(times)
print(len(expects))

200
1
200


In [ ]:
queries = {}
basecount = {}
g = gs[0]
#for g in gs:
for q in qs:
  q.reset()
  graphQL(q, g, 10)
  RI(q)
  totalresult = []
  count = 0
  QuikSI(q, g, {}, 1, totalresult)
  queries[q.graphid] = len(totalresult)
  basecount[q.graphid] = count
print(basecount)

{'query_dense_16_95.graph': 2408, 'query_dense_16_125.graph': 236, 'query_dense_16_11.graph': 2346, 'query_dense_16_26.graph': 146, 'query_dense_16_142.graph': 211, 'query_dense_16_186.graph': 178, 'query_dense_16_30.graph': 252, 'query_dense_16_179.graph': 1026, 'query_dense_16_74.graph': 157, 'query_dense_16_58.graph': 74, 'query_dense_16_191.graph': 135, 'query_dense_16_113.graph': 136, 'query_dense_16_62.graph': 120, 'query_dense_16_7.graph': 112, 'query_dense_16_51.graph': 212, 'query_dense_16_20.graph': 154, 'query_dense_16_168.graph': 318, 'query_dense_16_13.graph': 406, 'query_dense_16_35.graph': 449, 'query_dense_16_29.graph': 161, 'query_dense_16_195.graph': 61, 'query_dense_16_17.graph': 64, 'query_dense_16_155.graph': 675, 'query_dense_16_147.graph': 2371, 'query_dense_16_149.graph': 1044, 'query_dense_16_187.graph': 145, 'query_dense_16_139.graph': 611, 'query_dense_16_59.graph': 2498, 'query_dense_16_27.graph': 310, 'query_dense_16_122.graph': 286, 'query_dense_16_4.graph

In [ ]:
class PolicyNet(nn.Module):
  def __init__(self, dims):
    super(PolicyNet, self).__init__()
    self.gcns = nn.ModuleList([GCNConv(dims[i - 1], dims[i]) for i in range(1, len(dims) - 2)])
    self.mlps = nn.ModuleList([nn.Linear(dims[i - 1], dims[i]) for i in range(len(dims) - 2, len(dims))])

  def forward(self, x, edge_index):
    for gcn in self.gcns:
      x = gcn(x, edge_index)
    x = self.mlps[0](x)
    x = F.relu(x)
    x = self.mlps[1](x)
    return x

In [ ]:
class RL_QVO():
  def __init__(self, policynet, betaval, betah, posvalidreward, negvalidreward):
    self.policynet = policynet
    self.betaval = betaval
    self.betah = betah
    self.posvalidreward = posvalidreward
    self.negvalidreward = negvalidreward

  def train(self, q):
    step = len(q.node2label)

    steprewards = {}
    entropyrewards = {}
    log_probs = {}
    actions = {}
    for t in range(step):
      if t == 0:
        node = np.random.choice(range(step))
        q.phi.append(node)
        q.phiparent[node] = -1
        continue
      
      q.embeddings[t] = deepcopy(q.embeddings[0])
      
      mask = set()
      for node in q.node2label:
        q.embeddings[t][node][5] = step - t + 1
      
      for node in q.phi:
        q.embeddings[t][node][6] = 1
        mask |= q.edges[node]
      
      mask -= set(q.phi)
      mask = list(mask)
      if len(mask) == 1:
        q.phi.append(mask[0])
        for parent in q.phi:
          if parent in q.edges[mask[0]]:
            q.phiparent[mask[0]] = parent
            break
        continue

      x = q.embeddings[t].to(device)
      edge_index = q.edge_index.to(device)
      result = self.policynet(x, edge_index)
      maxindex = result.argmax().item()
      log_prob = F.softmax(result[mask], dim=0)

      if maxindex in mask:
        steprewards[t] = betaval * self.posvalidreward
      else:
        steprewards[t] = betaval * self.negvalidreward

      nplog = log_prob.t().detach().numpy()
      action = np.random.choice(nplog[0].shape[0], p=nplog[0])
      #action = log_prob.argmax().item()
      q.phi.append(mask[action])
      for parent in q.phi:
        if parent in q.edges[mask[action]]:
          q.phiparent[mask[action]] = parent
          break

      entropyreward = 0;
      for i in range(log_prob.shape[0]):
        if log_prob[i] > 0:
          entropyreward += (-log_prob[i]) * math.log(log_prob[i])
      entropyrewards[t] = betah * entropyreward
      log_probs[t] = log_prob
      actions[t] = action

    return steprewards, entropyrewards, log_probs, actions
    

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lr = 0.001
dims = [7, 32, 64, 32, 1]
policynet = PolicyNet(dims).to(device)
optimizer = optim.Adam(policynet.parameters(), lr=lr)
gamma = 0.9
betaval = 2
betah = 2
posvalidreward = 2
negvalidreward = -4 # absolute value should be larger than posvalidreward
clip = 0.1
epochs = 50
alphad = 2
alphal = 2
rlqvo = RL_QVO(policynet, betaval, betah, posvalidreward, negvalidreward)

In [ ]:
def train(qs, epochs):
  global count
  maxlen = 0
  for q in qs:
    q.reset()
    graphQL(q, g, 10)
    alphadegree = 0
    for node in q.candidateset:
      alphadegree = max(alphadegree, len(q.candidateset[node]))
    q.init_embedding(alphadegree, alphad, alphal, g)
    maxlen = max(maxlen, len(q.node2label) - 1)

  for epoch in range(epochs):    
    
    rewards = [0] * maxlen
    tmpprob = defaultdict(dict)
    oldprob = {}

    negcount = 0
    for q in qs:
      q.phi = []
      q.phiparent = {}
      steprewards, entropyrewards, log_probs, actions = rlqvo.train(q)
      totalresult = []
      count = 0

      QuikSI(q, g, {}, 1, totalresult)
      renum = count - basecount[q.graphid]
      if renum < 0:
        negcount += 1
      for t in range(maxlen):
        if renum > 0:
          renum = -math.log(renum)
        elif renum < 0:
          renum = math.log(-renum)
        else:
          renum = 0
        if t in steprewards:
          rewards[t] += math.pow(gamma, t + 1) * (renum + steprewards[t] + entropyrewards[t])
          tmpprob[q.graphid][t] = log_probs[t][actions[t]]
        else:
          rewards[t] = renum
          tmpprob[q.graphid][t] = torch.ones(1)

    print(negcount)

    losses = [0] * maxlen
    for q in qs:
      for t in tmpprob[q.graphid]:
        prob = tmpprob[q.graphid][t]
        if oldprob:
          losses[t] += min(prob / oldprob[q.graphid][t] * rewards[t], torch.clamp(prob / oldprob[q.graphid][t], 1.0 - clip, 1.0 + clip) * rewards[t])
        else:
          losses[t] += min(prob * rewards[t], torch.clamp(prob, 1.0 - clip, 1.0 + clip) * rewards[t])
    
    oldprob = tmpprob

    optimizer.zero_grad()
    loss = sum(losses) / len(qs)
    loss.backward()
    optimizer.step()
    print("Epoch: {0}, Loss: {1}， Rewards: {2}".format(epoch, loss.item(), sum(rewards).item() / len(qs)))

count = 0
train(random.sample(qs, 20), 10)
train(qs, 20)

6
Epoch: 0, Loss: -209.5333251953125， Rewards: -11.534709930419922
5
Epoch: 1, Loss: -280.9226989746094， Rewards: -15.559794616699218
9
Epoch: 2, Loss: -277.1666259765625， Rewards: -15.161428833007813
9
Epoch: 3, Loss: -304.9398193359375， Rewards: -16.73919219970703
8
Epoch: 4, Loss: -207.1382293701172， Rewards: -11.327440643310547
8
Epoch: 5, Loss: -248.55313110351562， Rewards: -13.622523498535156
8
Epoch: 6, Loss: -314.2757263183594， Rewards: -17.23102264404297
10
Epoch: 7, Loss: -459.470458984375， Rewards: -25.006153869628907
11
Epoch: 8, Loss: -301.82757568359375， Rewards: -16.409774780273438
7
Epoch: 9, Loss: -258.6339111328125， Rewards: -14.02100067138672
76
Epoch: 0, Loss: -680.8646240234375， Rewards: -3.6920266723632813
74
Epoch: 1, Loss: -401.4454650878906， Rewards: -2.1790840148925783
80
Epoch: 2, Loss: -675.1590576171875， Rewards: -3.63631103515625
72
Epoch: 3, Loss: -436.1169128417969， Rewards: -2.344932403564453
81
Epoch: 4, Loss: -507.0707092285156， Rewards: -2.7317205810

In [ ]:
fastercount = 0
fasterfreq = 0
for q in qs:
  q.reset()
  graphQL(q, g, 10)
  alphadegree = 0
  for node in q.candidateset:
    alphadegree = max(alphadegree, len(q.candidateset[node]))
  q.init_embedding(alphadegree, alphad, alphal, g)
  graphQL(q, g, 10)
  steprewards, entropyrewards, log_probs, actions = rlqvo.train(q)
  totalresult = []
  count = 0
  QuikSI(q, g, {}, 1, totalresult)
  if len(totalresult) == queries[q.graphid]:
    if count < basecount[q.graphid]:
      fasterfreq += 1
    fastercount += basecount[q.graphid] - count
    print("Q: {0}, Baseline: {1}, RLQVO: {2}".format(q.graphid, basecount[q.graphid], count))
  else:
    print("Q: {0} is wrong".format(q.graphid))

print("RLQVO total faster percentage: {0}, total faster count: {1}".format(fasterfreq / len(qs), fastercount))

Q: query_dense_16_95.graph, Baseline: 2408, RLQVO: 1524
Q: query_dense_16_125.graph, Baseline: 236, RLQVO: 190
Q: query_dense_16_11.graph, Baseline: 2346, RLQVO: 1702
Q: query_dense_16_26.graph, Baseline: 146, RLQVO: 1664
Q: query_dense_16_142.graph, Baseline: 211, RLQVO: 487
Q: query_dense_16_186.graph, Baseline: 178, RLQVO: 81
Q: query_dense_16_30.graph, Baseline: 252, RLQVO: 190
Q: query_dense_16_179.graph, Baseline: 1026, RLQVO: 1871
Q: query_dense_16_74.graph, Baseline: 157, RLQVO: 1302
Q: query_dense_16_58.graph, Baseline: 74, RLQVO: 35
Q: query_dense_16_191.graph, Baseline: 135, RLQVO: 42
Q: query_dense_16_113.graph, Baseline: 136, RLQVO: 154
Q: query_dense_16_62.graph, Baseline: 120, RLQVO: 114
Q: query_dense_16_7.graph, Baseline: 112, RLQVO: 66
Q: query_dense_16_51.graph, Baseline: 212, RLQVO: 2737
Q: query_dense_16_20.graph, Baseline: 154, RLQVO: 517
Q: query_dense_16_168.graph, Baseline: 318, RLQVO: 2816
Q: query_dense_16_13.graph, Baseline: 406, RLQVO: 6292
Q: query_dense_1

In [ ]:
def bfs(q):
  visited = set()
  queue = []
  nodes = [i for i in range(len(q.node2label))]
  node = random.choice(nodes)
  queue.append(node)
  visited.add(node)
  q.phiparent[node] = -1
  q.phi.append(node)
  while queue:
    node = queue[0]
    if node not in visited:
      q.phi.append(node)
    queue.pop(0)
    visited.add(node)
    for neighbor in q.edges[node]:
      if neighbor not in visited:
        q.phiparent[neighbor] = node
        queue.append(neighbor)


fastercount = 0
fasterfreq = 0
for q in qs:
  q.reset()
  graphQL(q, g, 10)
  
  bfs(q)

  totalresult = []
  count = 0
  QuikSI(q, g, {}, 1, totalresult)
  if len(totalresult) == queries[q.graphid]:
    if count < basecount[q.graphid]:
      fasterfreq += 1
    fastercount += basecount[q.graphid] - count
    print("Q: {0}, Baseline: {1}, Random: {2}".format(q.graphid, basecount[q.graphid], count))
  else:
    print("Q: {0} is wrong".format(q.graphid))

print("Random total faster percentage: {0}, total faster count: {1}".format(fasterfreq / len(qs), fastercount))

Q: query_dense_16_95.graph, Baseline: 2408, Random: 5638
Q: query_dense_16_125.graph, Baseline: 236, Random: 244
Q: query_dense_16_11.graph, Baseline: 2346, Random: 1083
Q: query_dense_16_26.graph, Baseline: 146, Random: 218
Q: query_dense_16_142.graph, Baseline: 211, Random: 89
Q: query_dense_16_186.graph, Baseline: 178, Random: 56
Q: query_dense_16_30.graph, Baseline: 252, Random: 395
Q: query_dense_16_179.graph, Baseline: 1026, Random: 2482
Q: query_dense_16_74.graph, Baseline: 157, Random: 298
Q: query_dense_16_58.graph, Baseline: 74, Random: 49
Q: query_dense_16_191.graph, Baseline: 135, Random: 104
Q: query_dense_16_113.graph, Baseline: 136, Random: 216
Q: query_dense_16_62.graph, Baseline: 120, Random: 131
Q: query_dense_16_7.graph, Baseline: 112, Random: 1021
Q: query_dense_16_51.graph, Baseline: 212, Random: 510
Q: query_dense_16_20.graph, Baseline: 154, Random: 71
Q: query_dense_16_168.graph, Baseline: 318, Random: 1052
Q: query_dense_16_13.graph, Baseline: 406, Random: 1401
